In [ ]:
import os
import re
import h5py
import pickle
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

import numpy as np

In [ ]:
print(plt.style.available)
plt.style.use('seaborn-v0_8-whitegrid')

In [ ]:
eig_values = pickle.load(open('/develop/results/eig/eigen_values_all.pkl', 'rb'))
eig_vect_rec, eig_vect_source = pickle.load(open('/develop/results/eig/eigen_vectors_all.pkl', 'rb'))

In [ ]:
# Plot the magnitude of the eigen values
fig, ax = plt.subplots(1,2,figsize=(12,5))
ax[0].plot(np.abs(eig_values))
ax[0].set_yscale('log')
ax[0].set_ylabel("|Eigen values|")
ax[0].set_xlabel("Mode number")

ax[1].plot(np.abs(eig_values))
ax[1].set_yscale('log')
ax[1].set_xscale('log')
ax[1].set_ylabel("|Eigen values|")
ax[1].set_xlabel("Mode number")
plt.tight_layout()

In [ ]:
S = np.sum(np.abs(eig_values)**2)
print(f"Sum rule = {S}")

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(8,5))
ax.plot(np.cumsum(np.abs(eig_values[0:50])**2)/S)
ax.set_xlabel("Mode numer")
ax.set_ylabel("Cumulative sum of eigenvalues")

## Mode projections

In [ ]:
path_meep_data = '/develop/data/'
folders = os.listdir(path_meep_data)
folders.sort()

In [ ]:
# Load in the fields
fields = []
# Only include directories that end with a number
numeric_dirs = [
    os.path.join(path_meep_data, d) 
    for d in folders 
    if os.path.isdir(os.path.join(path_meep_data, d)) and re.match(r'^\d+$', d)
]
#print(numeric_dirs)
# Iterate through numeric directories and read H5 files
# This can take a lot of time on Nautilus - moving data on disk (remote) to memory
for d in numeric_dirs:
    print(d)
    for fname in os.listdir(d):
        if re.match(r'^dft_\d+\.h5$', fname):
            h5_path = os.path.join(d, fname)
            with h5py.File(h5_path, 'r') as f:
                # Append the complex valued field
                # Only the final slice
                fields.append(f['ey_2.r'][:,:,-1] + 1j*f['ey_2.i'][:,:,-1])

In [ ]:
pickle.dump(fields, open('/develop/data/fields.pkl', 'wb'))

In [ ]:
# Flatten the fields
fields = [i.flatten() for i in fields]

In [ ]:
projections = []
for i, field in enumerate(fields):
    projections.append(np.conj(eig_vect_rec[:, 0:1000].T) @ field)
    print(i)

In [ ]:
pickle.dump(projections, open('/develop/results/projections_0.pkl', 'wb'))

In [ ]:
# Test some reconstructions
reconstructions = []
for i, p in enumerate(projections):
    num_modes = p.shape[-1]
    # Get the correct number of eigenvectors
    eig = eig_vect_rec[:,0:num_modes]
    # Reconstruct
    reconstructions.append(np.asarray(eig @ p))
    if i == 2:
        break

In [ ]:
fig, ax = plt.subplots(2, 3, figsize=(12, 8))

f = fields[0].reshape(166,166)
im0 = ax[0][0].imshow(f.real, cmap='viridis')
divider = make_axes_locatable(ax[0][0])
cax = divider.append_axes('right', size='5%', pad=0.05)
cbar = fig.colorbar(im0, cax=cax, orientation='vertical')
cbar.ax.tick_params(labelsize=14)
ax[0][0].set_title("Truth (real)")


im1 = ax[1][0].imshow(f.imag, cmap='viridis')
divider = make_axes_locatable(ax[1][0])
cax = divider.append_axes('right', size='5%', pad=0.05)
cbar = fig.colorbar(im1, cax=cax, orientation='vertical')
cbar.ax.tick_params(labelsize=14)
ax[1][0].set_title("Truth (imag)")

r = reconstructions[0].reshape(166,166)
im2 = ax[0][1].imshow(r.real, cmap='viridis')
divider = make_axes_locatable(ax[0][1])
cax = divider.append_axes('right', size='5%', pad=0.05)
cbar = fig.colorbar(im2, cax=cax, orientation='vertical')
cbar.ax.tick_params(labelsize=14)
ax[0][1].set_title("Recon (real)")


im3 = ax[1][1].imshow(r.imag, cmap='viridis')
divider = make_axes_locatable(ax[1][1])
cax = divider.append_axes('right', size='5%', pad=0.05)
cbar = fig.colorbar(im3, cax=cax, orientation='vertical')
cbar.ax.tick_params(labelsize=14)
ax[1][1].set_title("Recon (imag)")


diff_real = f.real - r.real
diff_imag = f.imag - r.imag

im4 = ax[0][2].imshow(diff_real, cmap='viridis')
divider = make_axes_locatable(ax[0][2])
cax = divider.append_axes('right', size='5%', pad=0.05)
cbar = fig.colorbar(im4, cax=cax, orientation='vertical')
cbar.ax.tick_params(labelsize=14)
ax[0][2].set_title("Diff (real)")

im5 = ax[1][2].imshow(diff_imag, cmap='viridis')
divider = make_axes_locatable(ax[1][2])
cax = divider.append_axes('right', size='5%', pad=0.05)
cbar = fig.colorbar(im5, cax=cax, orientation='vertical')
cbar.ax.tick_params(labelsize=14)
ax[1][2].set_title("Diff (imag)")

for a in ax.flatten():
    a.axis('off')

## Now, I want to test source eigenvectors evaluated at the receive plane

In this case, we take the source eigenvectors and evaluate them at the receive plane. This creates the projection of the source eigenvectors on the receive plan.

Then, we take the field (which is sampled at the recieve plan) and decompose it into the projected modes.

This should be equivalent to decomposing the fields into the reciever eigenvectors, but when I try and plot the receiver eigenvectors, they are jibberish.

However, plotting the projected source modes produces 'qualitatively good' modes.

I do the source projections in src/utils.py, and called in main.py

In [ ]:
source_projections = pickle.load(open('/develop/results/evaluated_modes_all.pkl', 'rb'))

In [ ]:
modes = [i.reshape(166,166) for i in source_projections]
print(len(modes))

In [ ]:
fig, ax = plt.subplots(3,3, figsize=(15,15))

for i,mode in enumerate(modes[0:9]):
    im1 = ax[i//3][i%3].imshow(mode.real, cmap='jet')
    ax[i//3][i%3].set_title(f"Mode {i}", fontsize=18)
    ax[i//3][i%3].grid(False)
    ax[i//3][i%3].axis('off')
    divider = make_axes_locatable(ax[i//3][i%3])
    cax = divider.append_axes('right', size='5%', pad=0.05)
    cbar = fig.colorbar(im1, cax=cax, orientation='vertical')
    cbar.ax.tick_params(labelsize=14)
    if i == 8:
        break
plt.tight_layout()


In [ ]:
source_projections.shape

In [ ]:
source_projections_decomp = []
for i, field in enumerate(fields):
    source_projections_decomp.append(source_projections[0:1000, :] @ field)

In [ ]:
source_projections_decomp = np.asarray(source_projections_decomp)

In [ ]:
source_projections_decomp.shape

In [ ]:
# Let's plot the magnitude of one decomp into the source projection
fig, ax = plt.subplots(1,2,figsize=(12,5))
ax[0].plot(np.abs(source_projections_decomp[0]))
ax[0].set_yscale('log')
ax[0].set_ylabel("|Field decomp into source projection|")
ax[0].set_xlabel("Mode number")

ax[1].plot(np.abs(source_projections_decomp[0]))
ax[1].set_yscale('log')
ax[1].set_xscale('log')
ax[1].set_ylabel("|Field decomp into source projection|")
ax[1].set_xlabel("Mode number")
plt.tight_layout()

In [ ]:
source_projections.shape

In [ ]:
# Okay, now lets build some reconstructions using a superposition of these modes and their projections
reconstructions_2 = []
for i, d in enumerate(source_projections_decomp):
    # Get the number of modes used
    temp = source_projections[0:num_modes]

    # Matrix multiply those modes by their decomp values
    temp = temp.T @ d

    # Append
    reconstructions_2.append(temp)

reconstructions_2 = [i.reshape(166,166) for i in reconstructions_2]

In [ ]:
fig, ax = plt.subplots(2, 3, figsize=(12, 8))

f = fields[0].reshape(166,166)
im0 = ax[0][0].imshow(f.real, cmap='viridis')
divider = make_axes_locatable(ax[0][0])
cax = divider.append_axes('right', size='5%', pad=0.05)
cbar = fig.colorbar(im0, cax=cax, orientation='vertical')
cbar.ax.tick_params(labelsize=14)
ax[0][0].set_title("Truth (real)")


im1 = ax[1][0].imshow(f.imag, cmap='viridis')
divider = make_axes_locatable(ax[1][0])
cax = divider.append_axes('right', size='5%', pad=0.05)
cbar = fig.colorbar(im1, cax=cax, orientation='vertical')
cbar.ax.tick_params(labelsize=14)
ax[1][0].set_title("Truth (imag)")

r = reconstructions_2[0].reshape(166,166)
im2 = ax[0][1].imshow(r.real, cmap='viridis')
divider = make_axes_locatable(ax[0][1])
cax = divider.append_axes('right', size='5%', pad=0.05)
cbar = fig.colorbar(im2, cax=cax, orientation='vertical')
cbar.ax.tick_params(labelsize=14)
ax[0][1].set_title("Recon (real)")


im3 = ax[1][1].imshow(r.imag, cmap='viridis')
divider = make_axes_locatable(ax[1][1])
cax = divider.append_axes('right', size='5%', pad=0.05)
cbar = fig.colorbar(im3, cax=cax, orientation='vertical')
cbar.ax.tick_params(labelsize=14)
ax[1][1].set_title("Recon (imag)")


diff_real = f.real - r.real
diff_imag = f.imag - r.imag

im4 = ax[0][2].imshow(diff_real, cmap='viridis')
divider = make_axes_locatable(ax[0][2])
cax = divider.append_axes('right', size='5%', pad=0.05)
cbar = fig.colorbar(im4, cax=cax, orientation='vertical')
cbar.ax.tick_params(labelsize=14)
ax[0][2].set_title("Diff (real)")

im5 = ax[1][2].imshow(diff_imag, cmap='viridis')
divider = make_axes_locatable(ax[1][2])
cax = divider.append_axes('right', size='5%', pad=0.05)
cbar = fig.colorbar(im5, cax=cax, orientation='vertical')
cbar.ax.tick_params(labelsize=14)
ax[1][2].set_title("Diff (imag)")

for a in ax.flatten():
    a.axis('off')